# 📊 Dataset Preprocessing for MiniGPT Instructivo

**Proyecto Final de Doctorado en IA/NLP**

Este notebook construye un dataset híbrido de instrucciones en español:
1. **Alpaca Español** (~52k instrucciones) - Base principal
2. **OpenAssistant Español** (~5k filtradas) - Conversaciones naturales
3. **Instrucciones originales** (espacio para tus 200-500 propias)

---

## 1. Setup e Instalación

In [ ]:
# Instalación de dependencias
!pip install datasets transformers pandas tqdm langdetect -q

print("✅ Dependencias instaladas")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
✅ Dependencias instaladas


In [ ]:
import pandas as pd
import json
import os
from datasets import load_dataset, Dataset, concatenate_datasets
from tqdm import tqdm
from collections import Counter
import re
from langdetect import detect, LangDetectException
import warnings
warnings.filterwarnings('ignore')

# Crear directorio para datos
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)

print("✅ Librerías cargadas")

✅ Librerías cargadas


## 2. Descarga de Datasets Base

### 2.1 Alpaca Español (Dataset Principal)

In [ ]:
# Opción 1: Alpaca Spanish directo
print("📥 Descargando Alpaca Español...")

try:
    # Intentar cargar versión en español directa
    alpaca_es = load_dataset("bertin-project/alpaca-spanish", split="train")
    print(f"✅ Alpaca Español cargado: {len(alpaca_es)} ejemplos")
except:
    try:
        # Alternativa: otra versión en español
        alpaca_es = load_dataset("josmunpen/alpaca-es", split="train")
        print(f"✅ Alpaca-ES cargado: {len(alpaca_es)} ejemplos")
    except:
        # Última opción: versión limpia en inglés (necesitará traducción)
        alpaca_es = load_dataset("yahma/alpaca-cleaned", split="train")
        print(f"⚠️ Cargado Alpaca inglés (necesita traducción): {len(alpaca_es)} ejemplos")

📥 Descargando Alpaca Español...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-3a8ceb2c27df89(…):   0%|          | 0.00/13.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51942 [00:00<?, ? examples/s]

✅ Alpaca Español cargado: 51942 ejemplos


In [ ]:
# Explorar estructura del dataset
print("\n📋 Estructura del dataset:")
print(alpaca_es)
print("\n📋 Columnas disponibles:")
print(alpaca_es.column_names)
print("\n📋 Ejemplo:")
print(alpaca_es[0])


📋 Estructura del dataset:
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 51942
})

📋 Columnas disponibles:
['instruction', 'input', 'output']

📋 Ejemplo:
{'instruction': '¿Qué significa DNA?', 'input': '', 'output': 'DNA significa ácido desoxirribonucleico.'}


### 2.2 OpenAssistant Español (Filtrado)

In [ ]:
print("📥 Descargando OpenAssistant...")
oasst = load_dataset("OpenAssistant/oasst1", split="train")
print(f"✅ OpenAssistant cargado: {len(oasst)} mensajes totales")

📥 Descargando OpenAssistant...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-b42a775f407cee(…):   0%|          | 0.00/39.5M [00:00<?, ?B/s]

data/validation-00000-of-00001-134b8fd0c(…):   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

✅ OpenAssistant cargado: 84437 mensajes totales


In [ ]:
# Filtrar solo mensajes en español
print("\n🔍 Filtrando mensajes en español...")

def is_spanish(text):
    """Detecta si el texto está en español"""
    try:
        if len(text) < 20:  # Textos muy cortos son difíciles de detectar
            return False
        return detect(text) == 'es'
    except LangDetectException:
        return False

# Filtrar por idioma español (usando el campo 'lang' si existe, sino detectamos)
if 'lang' in oasst.column_names:
    oasst_es = oasst.filter(lambda x: x['lang'] == 'es')
else:
    oasst_es = oasst.filter(lambda x: is_spanish(x['text']))

print(f"✅ Mensajes en español: {len(oasst_es)}")


🔍 Filtrando mensajes en español...


Filter:   0%|          | 0/84437 [00:00<?, ? examples/s]

✅ Mensajes en español: 22763


In [ ]:
# Explorar estructura de OpenAssistant
print("\n📋 Columnas de OpenAssistant:")
print(oasst_es.column_names)
print("\n📋 Ejemplo:")
print(oasst_es[0])


📋 Columnas de OpenAssistant:
['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels']

📋 Ejemplo:
{'message_id': '5249c721-e6d7-45f3-9680-f211d7be308e', 'parent_id': None, 'user_id': '63d9c326-9eef-46ae-92bf-1cfee1fa6128', 'created_date': '2023-02-06T02:16:52.167075+00:00', 'text': '¿CUales son las etapas del desarrollo y en qué consisten según Piaget?', 'role': 'prompter', 'lang': 'es', 'review_count': 3, 'review_result': True, 'deleted': False, 'rank': None, 'synthetic': False, 'model_name': None, 'detoxify': {'toxicity': 0.0003284301492385566, 'severe_toxicity': 0.00013771095837000757, 'obscene': 0.001163404667750001, 'identity_attack': 0.00021615914010908455, 'insult': 0.0006780127296224236, 'threat': 0.00013906563981436193, 'sexual_explicit': 8.358366903848946e-05}, 'message_tree_id': '5249c721-e6d7-45f3-9680-f211d7b

## 3. Conversión a Formato Alpaca

Formato estándar:
```json
{
    "instruction": "La tarea o pregunta",
    "input": "Contexto adicional (puede estar vacío)",
    "output": "La respuesta esperada"
}
```

In [ ]:
def normalize_alpaca_format(example, source_format='alpaca'):
    """
    Normaliza diferentes formatos al formato Alpaca estándar.
    """
    if source_format == 'alpaca':
        # Ya está en formato correcto, solo limpiamos
        return {
            'instruction': str(example.get('instruction', '')).strip(),
            'input': str(example.get('input', '')).strip(),
            'output': str(example.get('output', '')).strip()
        }
    return None

# Procesar Alpaca Español
print("🔄 Normalizando Alpaca Español...")
alpaca_normalized = []

for example in tqdm(alpaca_es):
    normalized = normalize_alpaca_format(example, 'alpaca')
    if normalized and normalized['instruction'] and normalized['output']:
        alpaca_normalized.append(normalized)

print(f"✅ Alpaca normalizado: {len(alpaca_normalized)} ejemplos válidos")

🔄 Normalizando Alpaca Español...


100%|██████████| 51942/51942 [00:01<00:00, 29826.72it/s]

✅ Alpaca normalizado: 51942 ejemplos válidos


In [ ]:
def convert_oasst_to_alpaca(dataset):
    """
    Convierte conversaciones de OpenAssistant a formato Alpaca.
    Agrupa mensajes por thread para crear pares instrucción-respuesta.
    """
    # Organizar por message_tree_id y parent_id
    messages_by_id = {msg['message_id']: msg for msg in dataset}

    alpaca_examples = []

    for msg in tqdm(dataset, desc="Convirtiendo OASST"):
        # Buscar mensajes que son respuestas (tienen parent)
        if msg.get('parent_id') and msg['parent_id'] in messages_by_id:
            parent = messages_by_id[msg['parent_id']]

            # El padre es la instrucción, el mensaje actual es la respuesta
            # Solo si el padre es del usuario (role == 'prompter')
            if parent.get('role') == 'prompter' and msg.get('role') == 'assistant':
                example = {
                    'instruction': parent['text'].strip(),
                    'input': '',
                    'output': msg['text'].strip()
                }

                # Filtrar ejemplos muy cortos o muy largos
                if (len(example['instruction']) > 10 and
                    len(example['output']) > 20 and
                    len(example['output']) < 2000):
                    alpaca_examples.append(example)

    return alpaca_examples

print("🔄 Convirtiendo OpenAssistant a formato Alpaca...")
oasst_alpaca = convert_oasst_to_alpaca(oasst_es)
print(f"✅ OpenAssistant convertido: {len(oasst_alpaca)} pares instrucción-respuesta")

🔄 Convirtiendo OpenAssistant a formato Alpaca...


Convirtiendo OASST: 100%|██████████| 22763/22763 [00:06<00:00, 3406.34it/s]

✅ OpenAssistant convertido: 14038 pares instrucción-respuesta


## 4. Limpieza y Normalización de Texto

In [ ]:
def clean_text(text):
    """
    Limpia y normaliza texto en español.
    """
    if not text:
        return ""

    # Convertir a string si no lo es
    text = str(text)

    # Eliminar espacios múltiples
    text = re.sub(r'\s+', ' ', text)

    # Eliminar espacios al inicio y final
    text = text.strip()

    # Normalizar comillas
    text = text.replace('"', '"').replace('"', '"')
    text = text.replace(''', "'").replace(''', "'")

    # Eliminar caracteres de control
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', text)

    return text

def clean_example(example):
    """Limpia un ejemplo completo"""
    return {
        'instruction': clean_text(example['instruction']),
        'input': clean_text(example['input']),
        'output': clean_text(example['output'])
    }

print("🧹 Limpiando textos...")
alpaca_clean = [clean_example(ex) for ex in tqdm(alpaca_normalized)]
oasst_clean = [clean_example(ex) for ex in tqdm(oasst_alpaca)]

print(f"✅ Limpieza completada")

🧹 Limpiando textos...


100%|██████████| 14038/14038 [00:01<00:00, 10110.71it/s]

✅ Limpieza completada


## 5. Espacio para Instrucciones Originales

**IMPORTANTE**: Aquí debes agregar tus 200-500 instrucciones originales.

Sugerencias de categorías:
- Ciencia y tecnología
- Explicaciones conceptuales
- Matemáticas y lógica
- Redacción y escritura
- Análisis crítico
- Tu área de especialización

In [ ]:
# Plantilla para instrucciones originales
# MODIFICA ESTA LISTA CON TUS PROPIAS INSTRUCCIONES

instrucciones_originales = [
    # === EJEMPLOS - REEMPLAZA CON LAS TUYAS ===
    {
        "instruction": "Explica qué es el aprendizaje por refuerzo como si le hablaras a un niño de 10 años.",
        "input": "",
        "output": "Imagina que tienes un perrito que está aprendiendo trucos. Cada vez que hace algo bien, le das una galleta (eso es una recompensa). Cada vez que hace algo mal, no le das nada. Con el tiempo, el perrito aprende qué cosas le dan galletas y las hace más seguido. El aprendizaje por refuerzo funciona igual: una computadora prueba muchas cosas, y cuando algo funciona bien, recibe una 'recompensa' virtual. Así aprende qué decisiones son buenas y cuáles no, igual que tu perrito aprendió sus trucos."
    },
    {
        "instruction": "¿Cuál es la diferencia entre machine learning y deep learning?",
        "input": "",
        "output": "Machine learning (aprendizaje automático) es un campo amplio donde las computadoras aprenden patrones de los datos sin ser programadas explícitamente. Deep learning (aprendizaje profundo) es un subconjunto de machine learning que usa redes neuronales con muchas capas. La diferencia clave es que en ML tradicional, los humanos diseñan las características que el modelo debe buscar, mientras que en deep learning, el modelo aprende automáticamente qué características son importantes. Deep learning requiere más datos y poder computacional, pero puede resolver problemas más complejos como reconocimiento de imágenes o procesamiento de lenguaje natural."
    },
    {
        "instruction": "Resume el concepto de tokenización en procesamiento de lenguaje natural.",
        "input": "",
        "output": "La tokenización es el proceso de dividir texto en unidades más pequeñas llamadas tokens. Estos tokens pueden ser palabras, subpalabras o caracteres. Es el primer paso en casi cualquier tarea de NLP porque las computadoras no pueden procesar texto directamente; necesitan números. Por ejemplo, la oración 'Hola mundo' se tokeniza en ['Hola', 'mundo']. Métodos modernos como BPE (Byte Pair Encoding) crean tokens de subpalabras, lo que permite manejar palabras desconocidas dividéndolas en partes conocidas."
    },
    {
        "instruction": "Escribe un poema corto sobre la inteligencia artificial.",
        "input": "",
        "output": "En circuitos de silicio late un sueño,\nde máquinas que aprenden a pensar,\ncon datos como estrellas en su empeño\nde patrones ocultos revelar.\n\nNo tiene corazón, pero comprende,\nno tiene ojos, pero logra ver,\nen cada línea de código se enciende\nun destello de nuevo amanecer."
    },
    {
        "instruction": "¿Qué es la atención en los modelos Transformer?",
        "input": "",
        "output": "La atención (attention) es un mecanismo que permite al modelo enfocarse en las partes más relevantes de la entrada al procesar cada elemento. En vez de tratar todas las palabras por igual, el modelo aprende a dar más 'peso' o importancia a ciertas palabras según el contexto. Por ejemplo, en la oración 'El gato se sentó en la alfombra porque estaba cansado', la atención ayuda al modelo a entender que 'estaba' se refiere al 'gato' y no a la 'alfombra'. La self-attention permite que cada palabra mire a todas las demás palabras de la secuencia para entender mejor su significado."
    },
    # === AGREGA MÁS INSTRUCCIONES AQUÍ ===
    # Objetivo: mínimo 200, ideal 500
]

print(f"📝 Instrucciones originales: {len(instrucciones_originales)}")
print(f"⚠️  Meta mínima: 200 instrucciones")
print(f"⭐ Meta ideal: 500 instrucciones")

📝 Instrucciones originales: 5
⚠️  Meta mínima: 200 instrucciones
⭐ Meta ideal: 500 instrucciones


In [ ]:
# Opción alternativa: Cargar instrucciones desde archivo JSON
# Si prefieres escribir tus instrucciones en un archivo separado

INSTRUCCIONES_FILE = 'data/raw/instrucciones_originales.json'

def cargar_instrucciones_desde_archivo():
    """Carga instrucciones originales desde un archivo JSON"""
    if os.path.exists(INSTRUCCIONES_FILE):
        with open(INSTRUCCIONES_FILE, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

def guardar_plantilla_instrucciones():
    """Guarda una plantilla para que escribas tus instrucciones"""
    plantilla = [
        {
            "instruction": "[ESCRIBE TU INSTRUCCIÓN AQUÍ]",
            "input": "",
            "output": "[ESCRIBE LA RESPUESTA ESPERADA AQUÍ]"
        }
    ] * 10  # 10 plantillas vacías para empezar

    with open(INSTRUCCIONES_FILE, 'w', encoding='utf-8') as f:
        json.dump(plantilla, f, ensure_ascii=False, indent=2)

    print(f"✅ Plantilla guardada en: {INSTRUCCIONES_FILE}")
    print("   Edita este archivo con tus instrucciones originales")

# Descomentar para crear plantilla:
# guardar_plantilla_instrucciones()

# Descomentar para cargar desde archivo:
# instrucciones_adicionales = cargar_instrucciones_desde_archivo()
# instrucciones_originales.extend(instrucciones_adicionales)

## 6. Combinación del Dataset Híbrido

In [ ]:
# Limpiar instrucciones originales
originales_clean = [clean_example(ex) for ex in instrucciones_originales]

# Combinar todos los datasets
print("\n🔗 Combinando datasets...")
print(f"   Alpaca Español: {len(alpaca_clean)}")
print(f"   OpenAssistant ES: {len(oasst_clean)}")
print(f"   Originales: {len(originales_clean)}")

dataset_completo = alpaca_clean + oasst_clean + originales_clean

print(f"\n✅ Dataset total: {len(dataset_completo)} ejemplos")


🔗 Combinando datasets...
   Alpaca Español: 51942
   OpenAssistant ES: 14038
   Originales: 5

✅ Dataset total: 65985 ejemplos


In [ ]:
# Eliminar duplicados
print("\n🔍 Eliminando duplicados...")

def get_example_hash(example):
    """Crea un hash para detectar duplicados"""
    return hash(example['instruction'].lower()[:100])

seen_hashes = set()
dataset_unico = []

for example in dataset_completo:
    h = get_example_hash(example)
    if h not in seen_hashes:
        seen_hashes.add(h)
        dataset_unico.append(example)

duplicados_eliminados = len(dataset_completo) - len(dataset_unico)
print(f"✅ Duplicados eliminados: {duplicados_eliminados}")
print(f"✅ Dataset final: {len(dataset_unico)} ejemplos únicos")


🔍 Eliminando duplicados...
✅ Duplicados eliminados: 8514
✅ Dataset final: 57471 ejemplos únicos


## 7. Análisis del Dataset

In [ ]:
# Estadísticas básicas
print("\n📊 ANÁLISIS DEL DATASET")
print("=" * 50)

# Longitudes
instruction_lengths = [len(ex['instruction']) for ex in dataset_unico]
output_lengths = [len(ex['output']) for ex in dataset_unico]

print(f"\n📏 Longitud de instrucciones:")
print(f"   Promedio: {sum(instruction_lengths)/len(instruction_lengths):.1f} caracteres")
print(f"   Mínimo: {min(instruction_lengths)} caracteres")
print(f"   Máximo: {max(instruction_lengths)} caracteres")

print(f"\n📏 Longitud de outputs:")
print(f"   Promedio: {sum(output_lengths)/len(output_lengths):.1f} caracteres")
print(f"   Mínimo: {min(output_lengths)} caracteres")
print(f"   Máximo: {max(output_lengths)} caracteres")


📊 ANÁLISIS DEL DATASET

📏 Longitud de instrucciones:
   Promedio: 71.8 caracteres
   Mínimo: 9 caracteres
   Máximo: 5189 caracteres

📏 Longitud de outputs:
   Promedio: 348.8 caracteres
   Mínimo: 1 caracteres
   Máximo: 3879 caracteres


In [ ]:
# Distribución de longitudes (visualización simple)
def print_histogram(data, title, bins=10):
    """Histograma ASCII simple"""
    min_val, max_val = min(data), max(data)
    bin_size = (max_val - min_val) / bins

    counts = [0] * bins
    for val in data:
        bin_idx = min(int((val - min_val) / bin_size), bins - 1)
        counts[bin_idx] += 1

    max_count = max(counts)

    print(f"\n{title}")
    print("-" * 50)

    for i, count in enumerate(counts):
        range_start = int(min_val + i * bin_size)
        range_end = int(min_val + (i + 1) * bin_size)
        bar_length = int(count / max_count * 30)
        bar = "█" * bar_length
        print(f"{range_start:5d}-{range_end:5d} | {bar} ({count})")

print_histogram(instruction_lengths, "📊 Distribución de longitud de instrucciones")
print_histogram(output_lengths, "📊 Distribución de longitud de outputs")


📊 Distribución de longitud de instrucciones
--------------------------------------------------
    9-  527 | ██████████████████████████████ (57362)
  527- 1045 |  (60)
 1045- 1563 |  (26)
 1563- 2081 |  (9)
 2081- 2599 |  (6)
 2599- 3117 |  (1)
 3117- 3635 |  (3)
 3635- 4153 |  (2)
 4153- 4671 |  (1)
 4671- 5189 |  (1)

📊 Distribución de longitud de outputs
--------------------------------------------------
    1-  388 | ██████████████████████████████ (36325)
  388-  776 | ████████████ (15100)
  776- 1164 | ███ (3689)
 1164- 1552 | █ (1484)
 1552- 1940 |  (693)
 1940- 2327 |  (127)
 2327- 2715 |  (35)
 2715- 3103 |  (10)
 3103- 3491 |  (4)
 3491- 3879 |  (4)


In [ ]:
# Palabras más frecuentes en instrucciones
print("\n🔤 Palabras más frecuentes en instrucciones:")

all_words = []
for ex in dataset_unico:
    words = re.findall(r'\b\w+\b', ex['instruction'].lower())
    all_words.extend(words)

# Filtrar stopwords básicas del español
stopwords_es = {'el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas', 'de', 'del',
                'a', 'en', 'que', 'y', 'es', 'para', 'por', 'con', 'se', 'su', 'al',
                'lo', 'como', 'más', 'o', 'pero', 'sus', 'le', 'ya', 'este', 'entre'}

filtered_words = [w for w in all_words if w not in stopwords_es and len(w) > 2]
word_counts = Counter(filtered_words)

print("\nTop 20 palabras:")
for word, count in word_counts.most_common(20):
    print(f"   {word}: {count}")


🔤 Palabras más frecuentes en instrucciones:

Top 20 palabras:
   oración: 6352
   siguiente: 5981
   sobre: 4002
   qué: 3457
   generar: 3439
   dado: 3434
   lista: 3355
   dos: 2993
   palabras: 2775
   crear: 2723
   cómo: 2629
   dada: 2576
   describe: 2294
   nombre: 2271
   crea: 2262
   explica: 2210
   tres: 2188
   escribe: 1974
   genera: 1922
   historia: 1866


In [ ]:
# Tipos de tareas detectadas (heurística simple)
print("\n📋 Tipos de tareas detectadas:")

task_patterns = {
    'explicación': r'\b(explica|describe|qué es|define|significa)\b',
    'generación': r'\b(escribe|genera|crea|redacta|compone)\b',
    'resumen': r'\b(resume|resumen|sintetiza|resumir)\b',
    'traducción': r'\b(traduce|traducir|traducción)\b',
    'clasificación': r'\b(clasifica|categoriza|tipo de)\b',
    'comparación': r'\b(compara|diferencia|similar|versus|vs)\b',
    'lista': r'\b(lista|enumera|menciona|cuáles son)\b',
    'código': r'\b(código|programa|función|script|python|java)\b',
    'matemáticas': r'\b(calcula|ecuación|fórmula|matemát)\b',
    'análisis': r'\b(analiza|análisis|evalúa|crítica)\b'
}

task_counts = Counter()

for ex in dataset_unico:
    instruction_lower = ex['instruction'].lower()
    for task_name, pattern in task_patterns.items():
        if re.search(pattern, instruction_lower):
            task_counts[task_name] += 1

for task, count in task_counts.most_common():
    percentage = count / len(dataset_unico) * 100
    print(f"   {task}: {count} ({percentage:.1f}%)")


📋 Tipos de tareas detectadas:
   generación: 6308 (11.0%)
   explicación: 5066 (8.8%)
   lista: 4184 (7.3%)
   clasificación: 1883 (3.3%)
   código: 1782 (3.1%)
   matemáticas: 774 (1.3%)
   comparación: 750 (1.3%)
   análisis: 723 (1.3%)
   resumen: 588 (1.0%)
   traducción: 86 (0.1%)


## 8. División Train/Validation y Guardado

In [ ]:
import random

# Mezclar aleatoriamente
random.seed(42)
random.shuffle(dataset_unico)

# División 95% train, 5% validation
split_idx = int(len(dataset_unico) * 0.95)

train_data = dataset_unico[:split_idx]
val_data = dataset_unico[split_idx:]

print(f"\n📊 División del dataset:")
print(f"   Training: {len(train_data)} ejemplos (95%)")
print(f"   Validation: {len(val_data)} ejemplos (5%)")


📊 División del dataset:
   Training: 54597 ejemplos (95%)
   Validation: 2874 ejemplos (5%)


In [ ]:
# Guardar en formato JSON
print("\n💾 Guardando datasets...")

with open('data/processed/train.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open('data/processed/validation.json', 'w', encoding='utf-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)

# Guardar dataset completo también
with open('data/processed/dataset_completo.json', 'w', encoding='utf-8') as f:
    json.dump(dataset_unico, f, ensure_ascii=False, indent=2)

print("✅ Archivos guardados:")
print("   - data/processed/train.json")
print("   - data/processed/validation.json")
print("   - data/processed/dataset_completo.json")


💾 Guardando datasets...
✅ Archivos guardados:
   - data/processed/train.json
   - data/processed/validation.json
   - data/processed/dataset_completo.json


In [ ]:
# Convertir a HuggingFace Dataset para uso con Transformers
print("\n🤗 Convirtiendo a formato HuggingFace Dataset...")

train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# Guardar en formato Arrow (más eficiente)
train_dataset.save_to_disk('data/processed/train_hf')
val_dataset.save_to_disk('data/processed/val_hf')

print("✅ Datasets HuggingFace guardados:")
print("   - data/processed/train_hf/")
print("   - data/processed/val_hf/")


🤗 Convirtiendo a formato HuggingFace Dataset...


Saving the dataset (0/1 shards):   0%|          | 0/54597 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2874 [00:00<?, ? examples/s]

✅ Datasets HuggingFace guardados:
   - data/processed/train_hf/
   - data/processed/val_hf/


## 9. Verificación Final

In [ ]:
# Verificar integridad
print("\n✅ VERIFICACIÓN FINAL")
print("=" * 50)

# Recargar y verificar
with open('data/processed/train.json', 'r', encoding='utf-8') as f:
    train_check = json.load(f)

print(f"\n📊 Resumen del dataset:")
print(f"   Total ejemplos: {len(dataset_unico)}")
print(f"   Training: {len(train_check)}")
print(f"   Validation: {len(val_data)}")

# Mostrar ejemplos aleatorios
print("\n📝 Ejemplos aleatorios del dataset:")
print("-" * 50)

for i, idx in enumerate(random.sample(range(len(train_check)), 3)):
    ex = train_check[idx]
    print(f"\n[Ejemplo {i+1}]")
    print(f"Instrucción: {ex['instruction'][:100]}...")
    print(f"Output: {ex['output'][:100]}...")


✅ VERIFICACIÓN FINAL

📊 Resumen del dataset:
   Total ejemplos: 57471
   Training: 54597
   Validation: 2874

📝 Ejemplos aleatorios del dataset:
--------------------------------------------------

[Ejemplo 1]
Instrucción: Proponga cinco preguntas para hacerle a un posible empleador durante una entrevista de trabajo...
Output: 1. ¿Cuáles son las responsabilidades clave de este puesto? 2. ¿Qué desafíos debo esperar enfrentar e...

[Ejemplo 2]
Instrucción: Describe la estructura y características predominantes de una novela de Ernest Hemingway....
Output: Ernest Hemingway es conocido por su estilo de escritura que se centra en un lenguaje sencillo que ca...

[Ejemplo 3]
Instrucción: Crea una oración que use la frase "por el contrario"....
Output: Por el contrario, creo que necesitamos tomar un enfoque diferente hacia la situación....


In [ ]:
# Resumen para el informe
print("\n" + "=" * 60)
print("📋 RESUMEN PARA EL INFORME TÉCNICO")
print("=" * 60)
print(f"""
Dataset Híbrido para MiniGPT Instructivo en Español
----------------------------------------------------

Composición:
- Alpaca Español: {len(alpaca_clean)} ejemplos
- OpenAssistant ES: {len(oasst_clean)} ejemplos
- Instrucciones originales: {len(originales_clean)} ejemplos

Después de limpieza y deduplicación:
- Total: {len(dataset_unico)} ejemplos únicos
- Training set: {len(train_data)} (95%)
- Validation set: {len(val_data)} (5%)

Estadísticas de texto:
- Longitud promedio instrucción: {sum(instruction_lengths)/len(instruction_lengths):.1f} caracteres
- Longitud promedio output: {sum(output_lengths)/len(output_lengths):.1f} caracteres

Formato: Alpaca (instruction, input, output)
Archivos generados: JSON y HuggingFace Dataset
""")


📋 RESUMEN PARA EL INFORME TÉCNICO

Dataset Híbrido para MiniGPT Instructivo en Español
----------------------------------------------------

Composición:
- Alpaca Español: 51942 ejemplos
- OpenAssistant ES: 14038 ejemplos  
- Instrucciones originales: 5 ejemplos

Después de limpieza y deduplicación:
- Total: 57471 ejemplos únicos
- Training set: 54597 (95%)
- Validation set: 2874 (5%)

Estadísticas de texto:
- Longitud promedio instrucción: 71.8 caracteres
- Longitud promedio output: 348.8 caracteres

Formato: Alpaca (instruction, input, output)
Archivos generados: JSON y HuggingFace Dataset



In [ ]:
# === GUARDAR DATOS EN GOOGLE DRIVE ===
from google.colab import drive
import shutil
import os

# Montar Google Drive
drive.mount('/content/drive')

# Crear carpeta del proyecto
project_path = '/content/drive/MyDrive/MiniGPT_Doctoral'
os.makedirs(f'{project_path}/data/processed', exist_ok=True)

# Copiar archivos
shutil.copy('data/processed/train.json', f'{project_path}/data/processed/')
shutil.copy('data/processed/validation.json', f'{project_path}/data/processed/')
shutil.copy('data/processed/dataset_completo.json', f'{project_path}/data/processed/')

# Copiar carpetas HuggingFace si existen
if os.path.exists('data/processed/train_hf'):
    shutil.copytree('data/processed/train_hf', f'{project_path}/data/processed/train_hf', dirs_exist_ok=True)
if os.path.exists('data/processed/val_hf'):
    shutil.copytree('data/processed/val_hf', f'{project_path}/data/processed/val_hf', dirs_exist_ok=True)

print("✅ ¡Archivos guardados en Google Drive!")
print(f"📁 Ubicación: {project_path}/data/processed/")

# Verificar
!ls -la {project_path}/data/processed/

Mounted at /content/drive
✅ ¡Archivos guardados en Google Drive!
📁 Ubicación: /content/drive/MyDrive/MiniGPT_Doctoral/data/processed/
total 58192
-rw------- 1 root root 29789736 Dec 12 21:51 dataset_completo.json
drwx------ 2 root root     4096 Dec 12 21:45 train_hf
-rw------- 1 root root 28294918 Dec 12 21:51 train.json
drwx------ 2 root root     4096 Dec 12 21:45 val_hf
-rw------- 1 root root  1494820 Dec 12 21:51 validation.json


---

## ✅ Siguiente Paso

El dataset está listo. Ahora procede al notebook `02_tokenizer_training.ipynb` para entrenar el tokenizador BPE.

**Recuerda**: Antes de continuar, agrega más instrucciones originales (meta: 200-500) en la Sección 5.